In [3]:
import argparse
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from tqdm.autonotebook import tqdm, trange

In [4]:
def get_embedding_function():
    embedding_model = "mixedbread-ai/mxbai-embed-large-v1"
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model,)
    return embeddings

In [6]:
CHROMA_PATH = "chroma"
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [7]:
def query_rag(query_text: str):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    model = Ollama(model="mistral")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [10]:
def main():
    # Create CLI.
    # parser = argparse.ArgumentParser()
    # parser.add_argument("query_text", type=str, help="The query text.")
    # args = parser.parse_args()
    # query_text = args.query_text
    query_text = input(str("Enter question you want to ask ::: "))
    query_rag(query_text)

In [12]:
main()

Response:  Based on the provided context, here are some key points for you as a newbie to follow when it comes to managing money:

1. Prioritize building assets before spending on luxuries like big houses or cars. Avoid getting stuck in the "Rat Race."

2. Use financial pressure as a motivator to come up with new ways of making more money instead of dipping into savings or investments. This will help you increase your financial intelligence and ability to make more money.

3. Be disciplined and avoid dipping into your savings or investments to pay bills. Instead, look for ways to cut expenses and find deals that can help you build wealth.

4. Educate yourself about personal finance by finding an expert in the field or reading books on the subject. Avoid investing in things you don't understand and be wary of people who have no idea what they are talking about.

5. Pay yourself first by setting aside money for savings, investments, or other financial goals before spending on non-essenti